# 'Applied' Exercise

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
cable = pd.read_csv('./Data/training.csv', na_values=(-999, 6)) # value = 6 corresponds to refusal to answer, 6 nowhere else in data

# Adjust the Feature Set

In [4]:
def CleanCableData(df):
    
    # omitting irrelevant/redundant columns and singular dummies (age=1, class=poor(d and e))
    
    drop = ['YES', 'ID', 'age', 'class', 'tele_have', 'd', 'de'] 
    df['value'] = [(i - 3) for i in df['value']] # Normalize (-2 to +2)
    df = df[[col for col in df.columns if col not in drop]]
    df = df.dropna()
    
    return df

In [5]:
cable = CleanCableData(cable)

In [6]:
y = pd.DataFrame(cable['buy'])
X = cable[[col for col in cable.columns if col != 'buy']]

feature_names = X.columns
X = np.array(X)

In [7]:
# add a constant column

X_c = np.c_[np.ones(X.shape[0]), X]
feature_names = feature_names.insert(0, 'constant')

# Model Estimation & Display

In [8]:
# Extend the sklearn LogisticRegression class to return Standard Errors and T-statistics

class ExtLogisticRegression(LogisticRegression):
    
    def __init__(self):
        
        LogisticRegression.__init__(self)
            
    def tstat(self, estimate, se): return estimate / se
    
    def getStats(self, X, feature_names): # passing X again after fit() is ugly, but unavoidable without serious class amendment.
        
        '''Courtesy of: 
        # https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficients
        # The covariance matrix as given by: (X'VX)^-1

        # X: (n x k), X': (k x n)
        # V: (n x n)

        # -> X'VX: (k x k)'''

        n = len(X)
        predictions = np.matrix(self.predict_proba(X))

        # Initiate matrix of 0's, fill diagonal with each predicted observation's variance
        V = np.matrix(np.zeros(shape = (n, n)))

        p_no = predictions[:,0] # array of all p(no buy)
        p_yes = (predictions[:,1]).A1 # flattened array of all p(buy)

        np.fill_diagonal(V, np.multiply(p_no, p_yes)) # n X n 

        # Covariance matrix
        cov = np.linalg.inv(X.T * V * X)

        # Standard errors
        se = np.sqrt(np.diag(cov))
        
        # Check with E-Views output -> ~ok
        
        tstats = pd.Series(map(self.tstat, self.coef_,se))[0]
        
        summary = pd.DataFrame({'Coefficients': list(self.coef_[0]), 'SE': list(se), 't-stat': list(tstats)})
        summary = summary.set_index(feature_names)
        
        self.summary = summary

In [9]:
Model = ExtLogisticRegression()

In [10]:
Model.fit(X_c, y)

C:\Users\cjrdn_000\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ExtLogisticRegression()

In [11]:
Model.getStats(X_c, feature_names)

In [12]:
# Please see  for an Eviews comparison; though there may exist some minute differences in
# calculation (esp. for the constant term), we think our estimates are sufficiently to Eviews close to justify moving forward.

# Eviews claims that, since normality conditions only holds asymptotically, a reported t-statistic is erroneous.
# We were unable to find the Eviews derivation of a p-value in a reasonable amount of time; due to the similarity of coefficients
# and SEs across the model elsewhere, we will just assume that the p-values derived from E-views apply here.  We can spend some
# more time tracking down and copying Eviews method if it would be a valuable exercise.

summary = Model.summary
summary = summary.drop('t-stat', axis=1)

eviews_output = pd.read_csv('./eviews_output.csv')
summary['p-value'] = list(eviews_output['Prob\xa0\xa0']) # I have no idea...weird encoding issue

summary

,Coefficients,SE,p-value
constant,0.357277,0.300927,0.0246
age2,0.271506,0.113775,0.0229
age3,0.335084,0.115239,0.0057
age4,0.342480,0.114885,0.0043
age5,0.005747,0.132889,0.9307
age6,0.006027,0.129059,0.9280
ab,-2.624517,0.114740,0.0000
c1,-2.195010,0.082904,0.0000
c2,-0.828344,0.057754,0.0000
children,0.276937,0.063324,0.0001


# Applied-Like Process Improvement

In [13]:
# Quick notes:  As with the original Applied project, we don't have many sign priors outside of the obvious: 

    # negative sign on price
    # negative sign on income measures (low-class stigma, inferior good - entertainment)
    # negative sign on the price of compliments (price_mc)

# Weaker cases can be made for other beliefs:

    # negative sign on rent (lease agreement forbids installation)
    # negative sign on consumption of disimliar programming (high-brow BBC channels)
    # positive sign on consumption of similar programming (low-brow ITV & CH4)
    
# Elsewhere, cultural differences between the US and UK, as well as the existince of bi-directional multiple marginal effects
# within the same variables (kids want tv more than adults, but parents don't want them to watch it) preclude strong
# assumptions about the effect of other demographic information.

# ***This should probably be done in order of prior violation magnitude, but we attempt all drops simultaneously for brevity***

# A minor prior violation on the income measures of EMP_FT and EMP_HWIFE (redundant proxies for income?) suggests a drop of both

# Since program quality is likely proxying an effect like value and it is statistically insignificant, we attempt a drop

# Since we don't have strong priors about the importance of the stat. insignificant Race, we attempt a drop

# price_mc is also insignificant, but we have good reasons from theory to leave it in (price of compliment, correctly signed)

# sex will prove to be insignificant in later rounds of testing; also dropped here for brevity

In [14]:
# This should be a more general function than just CleanCable()...

applied_drop = ['emp_ft', 'emp_hwife', 'prog_qual', 'race', 'sex'] 

In [15]:
cable = cable[[col for col in cable.columns if col not in applied_drop]]

In [16]:
# This mess should all be abstracted out to functions

y = pd.DataFrame(cable['buy'])
X = cable[[col for col in cable.columns if col != 'buy']]

feature_names = X.columns
X = np.array(X)

X_c = np.c_[np.ones(X.shape[0]), X]
feature_names = feature_names.insert(0, 'constant')

In [17]:
Model = ExtLogisticRegression()
Model.fit(X_c, y)

C:\Users\cjrdn_000\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ExtLogisticRegression()

In [18]:
Model.getStats(X_c, feature_names)

In [19]:
eviews_output = pd.read_csv('./eviews_output_final_model.csv')

In [20]:
summary = Model.summary
summary = summary.drop('t-stat', axis=1)

eviews_output = pd.read_csv('./eviews_output_final_model.csv')
summary['p-value'] = list(eviews_output['Prob\xa0\xa0']) # weird encoding issue

summary

,Coefficients,SE,p-value
constant,0.591121,0.271937,0.0001
age2,0.306919,0.112436,0.0109
age3,0.339983,0.114043,0.0054
age4,0.311357,0.113095,0.0100
age5,-0.124448,0.129280,0.3484
age6,-0.205551,0.122091,0.1037
ab,-2.576737,0.112713,0.0000
c1,-2.149678,0.081383,0.0000
c2,-0.802514,0.056590,0.0000
children,0.202889,0.060750,0.0029


# Final Model Evaluation & Comments:

In [21]:
# Our only remaining prior violation was somewhat weak (we don't have strong beliefs about previous television consumption)
# Our only remaining statistical insignificance gripes are with an age dummy; we should test their joint significance, 
# but programming a Wald-test from scratch seems excessive; we just leave them in for simplicities sake.

FinalModel = Model

# Lets see how the model predicts w.r.t. the training data and hold out:

In [22]:
def getPhat(model, X):
    
    predictions = Model.predict_proba(X)
    predictions = [p[1] for p in predictions]
    
    return sum(predictions) / len(predictions)

In [23]:
# Report the average predict p(buy) and actual p(buy) across the training set

getPhat(FinalModel, X_c), y.mean()[0] # Looks good

(0.2983591190727778, 0.29842342342342343)

In [24]:
cable_holdout = pd.read_csv('./Data/holdout.csv', na_values=(-999, 6))

In [25]:
cable_holdout = CleanCableData(cable_holdout)

In [26]:
y_2 = pd.DataFrame(cable_holdout['buy'])
X_2 = cable_holdout[[col for col in cable_holdout.columns if col != 'buy']]

X_2 = X_2[[col for col in X_2.columns if col not in applied_drop]]

# add a constant column

X_2c = np.c_[np.ones(X_2.shape[0]), X_2]
feature_names = feature_names.insert(0, 'constant')

In [27]:
# Report the average predict p(buy) and actual p(buy) across the holdout

getPhat(FinalModel, X_2c), y_2.mean()[0]

(0.3071644327303411, 0.30978260869565216)

In [28]:
# Our final model looks like it predicts well across our hold-out
# All in all, we think that the Applied-like logisticRegression model is, at least somewhat, believable
# Moving on to what-if questions...

# Elasticity Simulation

In [29]:
simulated_cable = cable.copy()

y = simulated_cable['buy']
X = simulated_cable[[col for col in simulated_cable.columns if col != 'buy']]

In [30]:
desired_values = range(8, 16)
desired_values

range(8, 16)

In [ ]:
# This is ugly as hell; can probably be reduced to 2 or 3 reasonable funcs

MARKET_SIZE = 1000000
outcomes = {}
for n in desired_values:
    
    X['price'] = [n for itme in X['price']]
    probas = [p[1] for p in Model.predict_proba(X)]
    average_proba = sum(probas) / len(probas)
    aggregate_demand = MARKET_SIZE * average_proba
    
    try:
        
        change_in_demand_as_percent = (aggregate_demand - last_value) / last_value
        change_in_price_as_percent = (n - last_n) / last_n
        elasticity = change_in_demand_as_percent / change_in_price_as_percent
        
    except(NameError):
        
        elasticity = 'n/a'
    
    last_n = n
    last_value = aggregate_demand
    
    print('Price: ' + str(n) + ', aggregate demand: ' + str(aggregate_demand) + ', elasticity: ' + str(elasticity))  

In [ ]:
def DemandElasticity(D1, D2, P1, P2):
    
    return ((D2 - D1) / D1) / ((P2 - P1) / P1)

In [ ]:
def getSimulationRange(x, dx):
    
    # Accepts a series and a step: returns an ordered list ranging from the minimum to the maximum of the list
    # in the series, seperated by steps
    
    min_x = min(x)
    max_x = max(x)
    
    r = np.arange(min_x, max_x + dx, dx)
    
    return r